In [1]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
import time
from sqlalchemy import create_engine
import json
import pymongo
from pymongo import MongoClient

# Extract CSVs into DataFrames

In [2]:
yelp_file = "../Resources/yelp_data.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


In [3]:
# All columns
yelp_df.columns

Index(['Unnamed: 0', 'restaurant_name', 'restaurant_url', 'restaurant_rating',
       'price_cuisine', 'address1', 'address2'],
      dtype='object')

In [15]:
# restaurant_name check
yelp_df["restaurant_name"].value_counts()
#yelp_df["restaurant_name"].count()

Sandwiches Unlimited Lunch Box            2
L & L Deli & Catering                     1
The 53 Grill                              1
JA Spice Island Jerk and American Food    1
Carver’s                                  1
                                         ..
Malay                                     1
Tierras y Sabores Restaurant              1
Paisano’s Pizzeria                        1
Bryans Luncheonette                       1
Blossom Asian Bistro                      1
Name: restaurant_name, Length: 239, dtype: int64

# Dropping duplicate restaurant_name value

In [21]:
yelp_df1= yelp_df.drop_duplicates(subset=['restaurant_name'])
yelp_df1

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


# Dropping duplicate address1 value

In [24]:
# address1 check
yelp_df["address1"].value_counts()

330 S Salem St          3
1250 Sussex Turnpike    2
39 W Clinton St         2
459 Main St             2
6B S Warren St          1
                       ..
25 E Main St            1
242 E Union Tpke        1
42 Main St              1
517 E Rte 10            1
12 Elm St               1
Name: address1, Length: 235, dtype: int64

In [23]:
yelp_df2= yelp_df.drop_duplicates(subset=['address1'])
yelp_df2

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


# Dropping duplicate restaurant_name and also address1 value

In [25]:
yelp_df3= yelp_df1.drop_duplicates(subset=['address1'])
yelp_df3

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


In [33]:
# average rating
average_rating = yelp_df3["restaurant_rating"].sum()/yelp_df3["restaurant_name"].count()
print("average_rating")
print(average_rating)


average_rating
4.0405982905982905


# Clean DataFrame

In [50]:
# restaurant_rating check
yelp_df3["restaurant_rating"].count()

234

In [61]:
# There are different length for restaurant_name and address1 values.So,we need to fill the empty values.
new_yelp_df=yelp_df3.fillna(0)
new_yelp_df

,Unnamed: 0,restaurant_name,restaurant_url,restaurant_rating,price_cuisine,address1,address2
0,0,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces",1181 Sussex Tpke,"Randolph, NJ 07869"
1,1,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4.0,"$$American (New), Bars, Pizza",500 NJ-10,"Randolph, NJ 07869"
2,2,Missy’s Main Street Cafe,https://www.yelp.com/biz/missys-main-street-ca...,5.0,"$Breakfast & Brunch, Cafes",181 E Main St,"Randolph, NJ 07869"
3,3,The Corner Bistro,https://www.yelp.com/biz/the-corner-bistro-ran...,4.0,"$$Diners, Italian",477 NJ-10,"Randolph, NJ 07869"
4,4,Beanbury,https://www.yelp.com/biz/beanbury-succasunna?o...,5.0,"Coffee & Tea, Breakfast & Brunch",37 Route 10 E,"Randolph, NJ 07869"
...,...,...,...,...,...,...,...
235,235,Clean Juice,https://www.yelp.com/biz/clean-juice-morristow...,4.5,"Juice Bars & Smoothies, Acai Bowls, Wraps",68 South St,"Randolph, NJ 07869"
236,236,LongHorn Steakhouse,https://www.yelp.com/biz/longhorn-steakhouse-f...,3.5,"$$Steakhouses, American (Traditional), Barbeque",50-J International Dr S,"Randolph, NJ 07869"
237,237,Hunan Chinese Room,https://www.yelp.com/biz/hunan-chinese-room-mo...,3.5,$$Chinese,255 Speedwell Ave,"Randolph, NJ 07869"
238,238,Brenda Lee Restaurant,https://www.yelp.com/biz/brenda-lee-restaurant...,3.5,Mexican,15 E Blackwell St,"Randolph, NJ 07869"


# Store CSV data into a DataFrame

In [39]:
csv_file = "../Resources/google_data.csv"
google_data_df = pd.read_csv(csv_file)
google_data_df

,Unnamed: 0,restaurant_name,restaurant_url,place_id,price_level,restaurant_rating,address
0,0,Popeyes Louisiana Kitchen,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,Rockaway Townsquare Mall-301 Mt Hope Road - Su...
1,1,Buffalo Wild Wings,"<a href=""https://maps.google.com/maps/contrib/...",ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,"343 Mount Hope Avenue, Rockaway"
2,2,Auntie Anne's,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,"301 Mount Hope Avenue Suite 1048, Rockaway"
3,3,Tabor Road Tavern,"<a href=""https://maps.google.com/maps/contrib/...",ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,"510 Tabor Road, Morris Plains"
4,4,Subway,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,"559 East Main Street Store # 2, Denville"
...,...,...,...,...,...,...,...
355,355,Anthony & Sons Bakery & Italian Deli,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,"15 New Jersey 10, Succasunna"
356,356,Pizza Express,"<a href=""https://maps.google.com/maps/contrib/...",ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,"312 Howard Boulevard, Mount Arlington"
357,357,Shell,"<a href=""https://maps.google.com/maps/contrib/...",ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,"274 U.S. 46, Mine Hill Township"
358,358,Dunkin',"<a href=""https://maps.google.com/maps/contrib/...",ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,"274 Rte 46, Mine Hill Township"


In [40]:
# All columns
google_data_df.columns

Index(['Unnamed: 0', 'restaurant_name', 'restaurant_url', 'place_id',
       'price_level', 'restaurant_rating', 'address'],
      dtype='object')

In [42]:
# restaurant name check
google_data_df["restaurant_name"].value_counts()

Subway                                12
Dunkin'                               11
Starbucks                             10
Panera Bread                           5
Domino's Pizza                         4
                                      ..
Tito's Burritos & Wings Morristown     1
Five Guys                              1
Aikou restaurant                       1
Riviera Maya Mexican Restaurant        1
Casella's Pizzeria & Deli              1
Name: restaurant_name, Length: 245, dtype: int64

# Dropping duplicate restaurant_name value

In [44]:
google_data_df1= google_data_df.drop_duplicates(subset=['restaurant_name'])
google_data_df1

,Unnamed: 0,restaurant_name,restaurant_url,place_id,price_level,restaurant_rating,address
0,0,Popeyes Louisiana Kitchen,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,Rockaway Townsquare Mall-301 Mt Hope Road - Su...
1,1,Buffalo Wild Wings,"<a href=""https://maps.google.com/maps/contrib/...",ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,"343 Mount Hope Avenue, Rockaway"
2,2,Auntie Anne's,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,"301 Mount Hope Avenue Suite 1048, Rockaway"
3,3,Tabor Road Tavern,"<a href=""https://maps.google.com/maps/contrib/...",ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,"510 Tabor Road, Morris Plains"
4,4,Subway,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,"559 East Main Street Store # 2, Denville"
...,...,...,...,...,...,...,...
353,353,One Touch Design,NaN,ChIJh5xqAeZ1w4kRXoS2ysw1L7I,NaN,5.0,"210 Kings Highway #7, Landing"
355,355,Anthony & Sons Bakery & Italian Deli,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,"15 New Jersey 10, Succasunna"
356,356,Pizza Express,"<a href=""https://maps.google.com/maps/contrib/...",ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,"312 Howard Boulevard, Mount Arlington"
357,357,Shell,"<a href=""https://maps.google.com/maps/contrib/...",ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,"274 U.S. 46, Mine Hill Township"


In [43]:
# restaurant address check
google_data_df["address"].value_counts()

3 Speedwell Avenue, Morristown         6
40 West Park Place, Morristown         4
641 Shunpike Road, Chatham Township    4
1 Convent Road, Morristown             4
184 Columbia Turnpike, Florham Park    3
                                      ..
92 Parsippany Boulevard, Boonton       1
58 West, U.S. 46, Pine Brook           1
160 East Main Street, Rockaway         1
270 U.S. 206, Chester                  1
57 New Jersey 10 E, Succasunna         1
Name: address, Length: 274, dtype: int64

# Dropping duplicate restaurant address value

In [45]:
google_data_df2= google_data_df.drop_duplicates(subset=['address'])
google_data_df2

,Unnamed: 0,restaurant_name,restaurant_url,place_id,price_level,restaurant_rating,address
0,0,Popeyes Louisiana Kitchen,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,Rockaway Townsquare Mall-301 Mt Hope Road - Su...
1,1,Buffalo Wild Wings,"<a href=""https://maps.google.com/maps/contrib/...",ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,"343 Mount Hope Avenue, Rockaway"
2,2,Auntie Anne's,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,"301 Mount Hope Avenue Suite 1048, Rockaway"
3,3,Tabor Road Tavern,"<a href=""https://maps.google.com/maps/contrib/...",ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,"510 Tabor Road, Morris Plains"
4,4,Subway,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,"559 East Main Street Store # 2, Denville"
...,...,...,...,...,...,...,...
355,355,Anthony & Sons Bakery & Italian Deli,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,"15 New Jersey 10, Succasunna"
356,356,Pizza Express,"<a href=""https://maps.google.com/maps/contrib/...",ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,"312 Howard Boulevard, Mount Arlington"
357,357,Shell,"<a href=""https://maps.google.com/maps/contrib/...",ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,"274 U.S. 46, Mine Hill Township"
358,358,Dunkin',"<a href=""https://maps.google.com/maps/contrib/...",ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,"274 Rte 46, Mine Hill Township"


# Dropping duplicate restaurant address and restaurant_name value

In [46]:
google_data_df3= google_data_df1.drop_duplicates(subset=['address'])
google_data_df3

,Unnamed: 0,restaurant_name,restaurant_url,place_id,price_level,restaurant_rating,address
0,0,Popeyes Louisiana Kitchen,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,Rockaway Townsquare Mall-301 Mt Hope Road - Su...
1,1,Buffalo Wild Wings,"<a href=""https://maps.google.com/maps/contrib/...",ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,"343 Mount Hope Avenue, Rockaway"
2,2,Auntie Anne's,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,"301 Mount Hope Avenue Suite 1048, Rockaway"
3,3,Tabor Road Tavern,"<a href=""https://maps.google.com/maps/contrib/...",ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,"510 Tabor Road, Morris Plains"
4,4,Subway,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,"559 East Main Street Store # 2, Denville"
...,...,...,...,...,...,...,...
353,353,One Touch Design,NaN,ChIJh5xqAeZ1w4kRXoS2ysw1L7I,NaN,5.0,"210 Kings Highway #7, Landing"
355,355,Anthony & Sons Bakery & Italian Deli,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,"15 New Jersey 10, Succasunna"
356,356,Pizza Express,"<a href=""https://maps.google.com/maps/contrib/...",ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,"312 Howard Boulevard, Mount Arlington"
357,357,Shell,"<a href=""https://maps.google.com/maps/contrib/...",ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,"274 U.S. 46, Mine Hill Township"


# Clean DataFrame

In [62]:
# price_level check
google_data_df3["price_level"].value_counts()

2.0    118
1.0     47
3.0     20
4.0      2
Name: price_level, dtype: int64

In [63]:
# restaurant_rating check
google_data_df3["restaurant_rating"].value_counts()

4.3    38
4.5    37
4.2    37
4.4    36
4.6    18
4.1    15
4.7    13
4.0     9
3.6     4
3.8     4
3.7     3
5.0     3
3.9     3
4.8     3
3.4     2
2.5     1
3.0     1
3.1     1
3.5     1
3.2     1
2.9     1
3.3     1
Name: restaurant_rating, dtype: int64

In [65]:
new_google_data_df=google_data_df3.fillna(0)
new_google_data_df

,Unnamed: 0,restaurant_name,restaurant_url,place_id,price_level,restaurant_rating,address
0,0,Popeyes Louisiana Kitchen,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,Rockaway Townsquare Mall-301 Mt Hope Road - Su...
1,1,Buffalo Wild Wings,"<a href=""https://maps.google.com/maps/contrib/...",ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,"343 Mount Hope Avenue, Rockaway"
2,2,Auntie Anne's,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,"301 Mount Hope Avenue Suite 1048, Rockaway"
3,3,Tabor Road Tavern,"<a href=""https://maps.google.com/maps/contrib/...",ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,"510 Tabor Road, Morris Plains"
4,4,Subway,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,"559 East Main Street Store # 2, Denville"
...,...,...,...,...,...,...,...
353,353,One Touch Design,0,ChIJh5xqAeZ1w4kRXoS2ysw1L7I,0.0,5.0,"210 Kings Highway #7, Landing"
355,355,Anthony & Sons Bakery & Italian Deli,"<a href=""https://maps.google.com/maps/contrib/...",ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,"15 New Jersey 10, Succasunna"
356,356,Pizza Express,"<a href=""https://maps.google.com/maps/contrib/...",ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,"312 Howard Boulevard, Mount Arlington"
357,357,Shell,"<a href=""https://maps.google.com/maps/contrib/...",ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,"274 U.S. 46, Mine Hill Township"


# Create database connection

# Creating a Collection